In [1]:
!pwd
!ls
!rm -r *
!ls
!git clone 'https://github.com/giangbang/traffic-flow-simulator.git'

In [ ]:
!add-apt-repository ppa:sumo/stable -y
!apt-get update -y
!apt-get install sumo sumo-tools sumo-doc
!pip install traci

In [ ]:
import os
os.environ['SUMO_HOME'] = "/usr/share/sumo/"

In [ ]:
!python main.py --episode 1  

Loading configuration ... done.
Step #0.00 (0ms ?*RT. ?UPS, TraCI: 3ms, vehicles TOT 0 ACT 0 BUF 0)                      

added SUMO_HOME to tools directory
{'nogui': False, 'train': False, 'episode': 1, 'yellow_phase': 5, 'green_phase': 10, 'stop': False}
device available:  cpu
0 model loaded successfully
{'0': 8}
{'0': 3}
Episode 0
{'0': array([[2]], dtype=int64)}
1
{'0': array([[2]], dtype=int64)}
1
{'0': array([[2]], dtype=int64)}
2
{'0': array([[2]], dtype=int64)}
2
{'0': array([[2]], dtype=int64)}
2
{'0': array([[2]], dtype=int64)}
3
{'0': array([[2]], dtype=int64)}
3
{'0': array([[2]], dtype=int64)}
3
{'0': array([[2]], dtype=int64)}
4
{'0': array([[2]], dtype=int64)}
4
{'0': array([[0]], dtype=int64)}
4
{'0': array([[0]], dtype=int64)}
5
{'0': array([[0]], dtype=int64)}
5
{'0': array([[0]], dtype=int64)}
5
{'0': array([[2]], dtype=int64)}
6
{'0': array([[2]], dtype=int64)}
6
{'0': array([[2]], dtype=int64)}
6
{'0': array([[2]], dtype=int64)}
7
{'0': array([[2]], dtype=int64)}
